# Try MWAS with covariates prepared in various ways

In this version, we only use the SNPs (from new files) also in old files

# Pick the regions we will test

In [1]:
library(data.table)

prev_hits <- fread("
Chr     pos        old_z       old_p
11   38247902        -27.1535308285104       2.30024742330298e-162
2    47933357        18.3327793004811        4.53147699327216e-75
7    1987910         10.1255085321387        4.25759742738181e-24
7    1987896         10.1255085321387        4.25759742738189e-24
7    1987797         10.0241523897721        1.19379483108027e-23
7    1987778         10.0105851568856        1.36940148731912e-23
12   2194742         -10.0072634920486       1.41615523554955e-23
")


In [2]:
#df <- fread("09.5-OUT_matched_SNP_meth_cov_chunked_EXPANSE_a2.csv")

In [3]:
df <- fread("09-OUT_matched_SNP_meth_cov_a2.csv")

## Try original code

In [4]:
###### model: learn elastic net model on training data 
######---------Input: trainX, trainY
######---------Return: selected features and coefficents

# original
elastic.net <- function(trainX,trainY){
    if(nrow(trainX)!=length(trainY)){
            stop("Number of observations is differerent")
    } 

    # optimize alpha---mixing parameter  
    a <- 0.5
    search <- foreach(ai = a, .combine = rbind) %dopar% {
        cv.fit <- cv.glmnet(
                        trainX,
                        trainY,
                        nfold = 5,
                        type.measure = "mse",
                        paralle = TRUE,
                        alpha = ai
                        )
        data.frame(
                        cvm = min(cv.fit$cvm),
                        lambda = cv.fit$lambda.min,
                        alpha = ai
                        )
        } 
    cv.opt <- search[search$cvm == min(search$cvm),] 

        # fit model by optimized alpha and lambda
        yfit = glmnet(
        trainX,
        trainY,
        lambda = cv.opt$lambda,
        alpha = cv.opt$alpha
                )       
        idf <- coef(yfit)
        idx <- which(idf != 0)
        selectf <- data.frame(
                features = idf@Dimnames[[1]][idx], 
                coefs = idf [idx]
        )
}

# modified to use lambda 1se and appropriate cvm
elastic.net <- function(trainX,trainY){
    if(nrow(trainX)!=length(trainY)){
            stop("Number of observations is differerent")
    } 

    # optimize alpha---mixing parameter  
    a <- 0.5
    search <- foreach(ai = a, .combine = rbind) %dopar% {
        set.seed(42)
        cv.fit <- cv.glmnet(
                        trainX,
                        trainY,
                        nfold = 5,
                        type.measure = "mse",
                        paralle = TRUE,
                        alpha = ai
                        )
        print(paste0("Dim of trainX: ", dim(trainX)))
        print(paste0("Len of trainY: ", length(trainY)))
        coef_matrix <- as.matrix(coef(cv.fit))

        non_zero_coefs <- coef_matrix[coef_matrix != 0, , drop = FALSE]
        print("Coefficients when fitting: ")
        print(non_zero_coefs)

        data.frame(
                        cvm = cv.fit$cvm[cv.fit$lambda == cv.fit$lambda.1se],
                        lambda = cv.fit$lambda.1se,
                        alpha = ai
                        )
        } 
    cv.opt <- search[search$cvm == min(search$cvm),] 

        # fit model by optimized alpha and lambda
    set.seed(42)
    yfit <- glmnet(
    trainX,
    trainY,
    lambda = cv.opt$lambda,
    alpha = cv.opt$alpha)

    coef_matrix <- as.matrix(coef(yfit))

    non_zero_coefs <- coef_matrix[coef_matrix != 0, , drop = FALSE]
    print("Coefficients when optimal: ")
    print(non_zero_coefs)

    idf <- coef(yfit)
    idx <- which(idf != 0)
    selectf <- data.frame(
            features = idf@Dimnames[[1]][idx], 
            coefs = idf [idx]
    )
}

MWAS <- function(gwas, weight, geno){
        z <- gwas %*% weight
        z.cor <- cor(geno)
        se <- sqrt(weight %*%  z.cor %*%  weight)
        z <- z/se
        p=pnorm(abs(z),lower.tail=F)*2
        return(c(z, p))
}

In [5]:
df <- df[which(df$Chr == 7), ]

In [6]:
i <- 2

In [7]:
library("glmnet")
library("e1071")
library("doParallel")

set.seed(2018)
wind <- c(5000,10000)
# output directory
#outd <- "/dcl02/lieber/shan/shizhong/finemapping/GWAS/tags/scz3/mwas/chr22/1/"
outd <- "20-OUT_original_mwas_sanity_test/"

Loading required package: Matrix

Loaded glmnet 4.1-8

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



In [8]:
suppressWarnings(library(bsseq))

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:Matrix’:

    expand, unname


The following objects are masked from ‘package:data.table’:

    first, second


The following object is masked from ‘package:utils’:

    findMatches


The

In [9]:
# load data for mwas
# load("./rda/caudate_mwas_data_chr22.rda")
load(df$methylation_data[i])

p <- getMeth(BSobj2)


rownames(p) <- start(BSobj2)

sites_to_test <- which(start(BSobj2) >= (1987910 - 500) & start(BSobj2) <= (1987910 + 500))
p <- p[sites_to_test, ]

# candidate cg
cg <- as.numeric(rownames(p))

# regress out covariates
#load("covs_for_meqtl.rda")

In [10]:
sites_to_test

[1] 58792 58793 58794 58795 58796 58797 58798 58799 58800 58801 58802 58803
[13] 58804 58805 58806 58807 58808 58809 58810 58811 58812 58813 58814 58815
[25] 58816 58817 58818 58819 58820 58821 58822 58823 58824 58825 58826 58827
[37] 58828 58829 58830 58831 58832 58833 58834 58835 58836 58837 58838 58839
[49] 58840

In [11]:
covs <- fread(df$cov_file[i])
covs <- t(covs)
colnames(covs) <- covs[1, ]
covs <- covs[2:nrow(covs), ]
# transpose so we have same orientation as original code

In [12]:
BSobj2$brnum <- gsub("Br0", "Br", BSobj2$brnum)
colnames(covs) <- gsub("Br0", "Br", colnames(covs))

In [13]:
length(match(BSobj2$brnum,colnames(covs)))

[1] 297

In [14]:
mat <- match(BSobj2$brnum,colnames(covs)) 
covs <- t(covs[,mat])
p.residual=matrix(NA,dim(p)[1],dim(p)[2])

In [15]:
rownames(covs)[is.na(covs[, 'genoPC1'])] <- BSobj2$brnum[is.na(covs[, 'genoPC1'])]

In [16]:
colnames(p.residual) <- BSobj2$brnum

In [17]:
covs <- as.data.frame(covs)
# Convert all columns except Dx and Sex from character to numeric
cols_to_convert <- setdiff(names(covs), c("Dx", "Sex"))

for (col in cols_to_convert) {
  covs[[col]] <- as.numeric(covs[[col]])
}

# Print the modified data frame to check the conversion
#print(dat)


In [18]:
for (i in 1:dim(p)[1]) { # For each methylation site
    dat <- as.data.frame(cbind(y = p[i,], covs))
    
    # Check for rows with NAs (the ones for which we don't have covariate data)
    valid_rows <- complete.cases(dat)
    
    if (sum(valid_rows) > 0) {
        dat_valid <- dat[valid_rows,]
        model.res <- lm(y ~ ., data = dat_valid)
        
        # Store residuals in the corresponding positions
        p.residual[i, valid_rows] <- resid(model.res)
    }
}


# for(i in 1:dim(p)[1]){ # foro each methylation site
#         dat <- as.data.frame(cbind(p[i,],covs))
#         colnames(dat) <- c("y",paste0("x",1:ncol(covs)))
#         model.res <- lm(reformulate(paste0("x",1:ncol(covs)), "y"),dat)
#         p.residual[i,] = resid(model.res) 
# }

In [19]:
load("p1.rda")

In [20]:
wind <- 10000

In [21]:
# built predition models
idx.ea <- BSobj2$race == "CAUC"

In [22]:
snp2_sorted <- snp2[, order(names(snp2))]

In [23]:
colnames(snp2) <- gsub("Br0", "Br", colnames(snp2))

In [24]:
snp2 <- snp2[, colnames(snp2) %in% colnames(p.residual)]

In [25]:
dim(snp2)

[1]  56 297

In [26]:
snp2_positions <- stringr::str_split_fixed(rownames(snp2), ":", 3)[, 2]

## Test for one

In [27]:
dim(snp2)

[1]  56 297

In [28]:
rownames(snp2)

[1] "chr7:1963098:T:C" "chr7:1963408:C:T" "chr7:1964786:C:T" "chr7:1966112:T:C"
 [5] "chr7:1973362:G:A" "chr7:1975412:T:G" "chr7:1976335:C:T" "chr7:1977810:C:T"
 [9] "chr7:1978402:C:T" "chr7:1978783:G:T" "chr7:1979188:C:A" "chr7:1979235:G:A"
[13] "chr7:1980240:G:A" "chr7:1981269:G:C" "chr7:1981360:G:A" "chr7:1982428:C:G"
[17] "chr7:1985366:G:C" "chr7:1985405:A:G" "chr7:1987676:G:T" "chr7:1987689:G:A"
[21] "chr7:1987719:C:T" "chr7:1987813:T:C" "chr7:1989028:A:C" "chr7:1989333:G:T"
[25] "chr7:1989994:G:C" "chr7:1990232:T:C" "chr7:1990305:T:C" "chr7:1990372:G:C"
[29] "chr7:1990469:G:A" "chr7:1990652:G:T" "chr7:1993168:C:T" "chr7:1993230:A:T"
[33] "chr7:1994145:G:C" "chr7:1994382:C:T" "chr7:1994558:A:G" "chr7:1994714:G:A"
[37] "chr7:1994927:A:G" "chr7:1997034:T:G" "chr7:1997580:G:A" "chr7:1998126:G:A"
[41] "chr7:1999959:A:C" "chr7:2001797:A:G" "chr7:2002003:G:A" "chr7:2002813:C:T"
[45] "chr7:2002912:C:G" "chr7:2004461:G:C" "chr7:2005716:T:C" "chr7:2006553:G:C"
[49] "chr7:2006693:A:G" "chr7:2007195:C:T" "chr7:2008210:A:G" "chr7:2008240:G:A"
[53] "chr7:2008501:T:C" "chr7:2008585:G:C" "chr7:2008700:A:G" "chr7:2009071:A:G"

In [29]:
k <- 1

i <- 1

range1 <- ifelse(cg[i] - wind[k] > 0,cg[i] - wind[k],0)
range2 <- cg[i] + wind[k]

range1
range2

[1] 1977413

[1] 1997413

In [30]:
cat(i,"\n")
range1 <- ifelse(cg[i] - wind[k] > 0,cg[i] - wind[k],0)
range2 <- cg[i] + wind[k]
range1
range2

idx <- map2$POS > range1 & map2$POS < range2

length(idx)

1 


[1] 1977413

[1] 1997413

[1] 56

In [31]:
range1

[1] 1977413

In [32]:
range2

[1] 1997413

In [33]:
# go to next cg if no snps within window
if(sum(idx) <= 1){
        next
}
geno <- snp2[idx,]
rownames(geno) <- map2$POS[idx]
trainX <- t(geno)
trainY <- p.residual[i,]

In [34]:
dim(snp2)

dim(geno)

head(trainY[order(names(trainY))])

[1]  56 297

[1]  31 297

Br1003       Br1004       Br1007       Br1016       Br1017       Br1021 
 0.032737590  0.019424071  0.017150662  0.004945029 -0.007041670 -0.023944463

In [35]:
inspect <- t(snp2)
inspect <- inspect[order(rownames(inspect)), ]

In [36]:
fit <- elastic.net(trainX,trainY)

Warning message:
“executing %dopar% sequentially: no parallel backend registered”


[1] "Dim of trainX: 297" "Dim of trainX: 31" 
[1] "Len of trainY: 297"
[1] "Coefficients when fitting: "
                      s1
(Intercept) 3.388132e-20
[1] "Coefficients when optimal: "
                       s0
(Intercept) -2.577859e-18
1979188      2.816307e-18


In [37]:
fit

features,coefs
<chr>,<dbl>
(Intercept),-2.577859e-18
1979188,2.816307e-18


We're getting different elasticnet results here than with our package. Why?

Let's go inside elastic.net function

In [38]:
dim(trainX)

[1] 297  31

Deeper still

In [39]:
trainX2 <- readRDS("20-IN_sanity_test_Xmatrix.rds")

In [40]:
trainy2 <- readRDS("20-IN_sanity_test_y.rds")

In [41]:
fold_id <- readRDS("20-IN_sanity_test_fold_id.rds")

In [42]:
set.seed(42)
cv.fit <- cv.glmnet(
                trainX,
                trainY,
    nfold = 5,
                #foldid = fold_id,                    
        type.measure = "mse",
                    paralle = TRUE,
                    alpha = 0.5
                    )

In [43]:
cv.fit


Call:  cv.glmnet(x = trainX, y = trainY, type.measure = "mse", nfolds = 5,      parallel = TRUE, alpha = 0.5) 

Measure: Mean-Squared Error 

      Lambda Index   Measure        SE Nonzero
min 0.004499     5 0.0004443 3.844e-05       1
1se 0.006528     1 0.0004468 3.781e-05       0

In [44]:
coef(cv.fit)

32 x 1 sparse Matrix of class "dgCMatrix"
                      s1
(Intercept) 3.388132e-20
1977810     .           
1978402     .           
1978783     .           
1979188     .           
1979235     .           
1980240     .           
1981269     .           
1981360     .           
1982428     .           
1985366     .           
1985405     .           
1987676     .           
1987689     .           
1987719     .           
1987813     .           
1989028     .           
1989333     .           
1989994     .           
1990232     .           
1990305     .           
1990372     .           
1990469     .           
1990652     .           
1993168     .           
1993230     .           
1994145     .           
1994382     .           
1994558     .           
1994714     .           
1994927     .           
1997034     .           

In [45]:
head(fold_id)

[1] 4 3 4 3 1 2

In [46]:
a <- 0.5
search <- foreach(ai = a, .combine = rbind) %dopar% {
    set.seed(42)
    cv.fit <- cv.glmnet(
                trainX2,
                trainy2,
                foldid = fold_id,                    
        type.measure = "mse",
                    paralle = TRUE,
                    alpha = ai
                    )
    data.frame(
                    cvm = cv.fit$cvm[cv.fit$lambda == cv.fit$lambda.1se],
                    lambda = cv.fit$lambda.1se,
                    alpha = ai
                    )
    } 

In [47]:
cv.opt <- search[search$cvm == min(search$cvm),] 

In [48]:
cv.opt

,cvm,lambda,alpha
,<dbl>,<dbl>,<dbl>
1,0.0004224127,0.007649007,0.5


In [49]:
cv.fit


Call:  cv.glmnet(x = trainX, y = trainY, type.measure = "mse", nfolds = 5,      parallel = TRUE, alpha = 0.5) 

Measure: Mean-Squared Error 

      Lambda Index   Measure        SE Nonzero
min 0.004499     5 0.0004443 3.844e-05       1
1se 0.006528     1 0.0004468 3.781e-05       0

In [50]:
coef(cv.fit) # This matches what we get out of tune_alpha, without fitting optimized parameters on whole data

32 x 1 sparse Matrix of class "dgCMatrix"
                      s1
(Intercept) 3.388132e-20
1977810     .           
1978402     .           
1978783     .           
1979188     .           
1979235     .           
1980240     .           
1981269     .           
1981360     .           
1982428     .           
1985366     .           
1985405     .           
1987676     .           
1987689     .           
1987719     .           
1987813     .           
1989028     .           
1989333     .           
1989994     .           
1990232     .           
1990305     .           
1990372     .           
1990469     .           
1990652     .           
1993168     .           
1993230     .           
1994145     .           
1994382     .           
1994558     .           
1994714     .           
1994927     .           
1997034     .           

In [51]:
a <- 0.5

In [52]:
search <- foreach(ai = a, .combine = rbind) %dopar% {
    set.seed(42)
    cv.fit <- cv.glmnet(
                    trainX2,
                    trainy2,
                    foldid = fold_id,
                    type.measure = "mse",
                    paralle = TRUE,
                    alpha = ai
                    )
    data.frame(
                    cvm = cv.fit$cvm[cv.fit$lambda == cv.fit$lambda.1se],
                    lambda = cv.fit$lambda.1se,
                    alpha = ai
                    )
    } 
cv.opt <- search[search$cvm == min(search$cvm),] 

    # fit model by optimized alpha and lambda
    set.seed(42)
    yfit = glmnet(
    trainX2,
    trainy2,
    lambda = cv.opt$lambda,
    alpha = cv.opt$alpha
            )       
    idf <- coef(yfit)
    idx <- which(idf != 0)
    selectf <- data.frame(
            features = idf@Dimnames[[1]][idx], 
            coefs = idf [idx]
    )

In [53]:
cv.opt # GOOD! should be 
#cvm	lambda	alpha
#<dbl>	<dbl>	<dbl>
#1	0.0004224127	0.007649007	0.5


,cvm,lambda,alpha
,<dbl>,<dbl>,<dbl>
1,0.0004224127,0.007649007,0.5


In [54]:
search

cvm,lambda,alpha
<dbl>,<dbl>,<dbl>
0.0004224127,0.007649007,0.5


Make sure we have same results not only from fitting but for optimal model with fixed alpha, lambda also

In [55]:
selectf

features,coefs
<chr>,<dbl>
(Intercept),-4.062561e-05
chr7:1977810:C:T,7.759077e-04
chr7:1981360:G:A,-1.070675e-03
chr7:1987719:C:T,-1.224151e-03
chr7:1987813:T:C,1.768736e-04
chr7:1990232:T:C,-1.298076e-03
chr7:1994382:C:T,1.105434e-03


In [56]:
dim(trainX)

[1] 297  31

In [57]:
models.all <- c()

In [58]:
rbind(models.all, cv.fit)

,lambda,cvm,cvsd,cvup,cvlo,nzero,call,name,glmnet.fit,lambda.min,lambda.1se,index
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<language>,<chr>,<elnet>,<dbl>,<dbl>,"<int[,1]>"
cv.fit,0.006527....,0.000446....,3.781117....,0.000484....,0.000409....,"0, 1, 1,....",cv.glmne....,Mean-Squ....,c(s0 = 3....,0.004499....,0.006527....,"5, 1"


Is 1se vs min for lambda the problem?

## Run for all

In [59]:
dim(snp2)

[1]  56 297

In [60]:
dim(snp2)

[1]  56 297

In [61]:
snp3 <- readRDS("20-IN_check_SNP_window_pos1987413_win50000.rds")

In [62]:
dim(snp2)

[1]  56 297

In [63]:
snp3 <- t(snp3)

In [64]:
map3 <- data.frame(POS = stringr::str_split_fixed(rownames(snp3), ":", 3)[, 2])

In [65]:
elastic.net(trainX, trainY)

[1] "Dim of trainX: 297" "Dim of trainX: 31" 
[1] "Len of trainY: 297"
[1] "Coefficients when fitting: "
                      s1
(Intercept) 3.388132e-20
[1] "Coefficients when optimal: "
                       s0
(Intercept) -2.577859e-18
1979188      2.816307e-18


In [66]:
trainX[1:10, 1:10]

,1977810,1978402,1978783,1979188,1979235,1980240,1981269,1981360,1982428,1985366
Br836,0,0.008,0.000,0.000,2,2.000,2,2.000,2.000,2
Br845,1,2.000,2.000,0.000,2,2.000,2,2.000,2.000,2
Br848,1,1.991,1.990,0.010,2,1.905,2,1.006,1.006,2
Br863,0,0.000,0.000,0.000,2,2.000,2,2.000,2.000,2
Br914,0,1.981,1.979,0.047,2,2.000,2,2.000,2.000,2
Br948,0,0.024,0.024,1.001,2,2.000,2,2.000,2.000,2
Br949,2,2.000,2.000,1.997,2,2.000,2,0.005,2.000,2
Br963,0,0.015,0.019,1.997,2,2.000,2,2.000,2.000,2
Br983,1,2.000,2.000,0.001,1,1.000,1,1.000,1.000,1
Br991,0,0.000,0.000,1.000,2,2.000,2,2.000,2.000,2


In [67]:
trainY[1:10]

Br1122        Br2285        Br1764        Br1464        Br5062 
-0.0138154121 -0.0225516889 -0.0004894733  0.0340564405 -0.0329548084 
       Br1446        Br1503        Br1946        Br5323        Br1297 
 0.0244990255 -0.0075554592  0.0427314728  0.0199602498 -0.0177260869

In [68]:
p.residual <- p.residual[, order(colnames(p.residual))]

In [69]:
for(k in 1:length(wind)){
    models.ea <- c()
    models.all <- c()
    for(i in 1:length(cg)){
    #for(i in 1){
            cat(i,"\n")
            print(paste0("This cg is: ", cg[i]))
            range1 <- ifelse(cg[i] - wind[k] > 0,cg[i] - wind[k],0)
            range2 <- cg[i] + wind[k]
            idx <- map3$POS > range1 & map3$POS < range2
            # go to next cg if no snps within window
            if(sum(idx) <= 1){
                    next
            }
            geno <- snp3[idx,] # changed snp2 to snp3
            rownames(geno) <- map3$POS[idx]
            trainX <- t(geno)
            trainY <- p.residual[i,]
            fit <- tryCatch(
                    elastic.net(trainX,trainY),
                    error = function(e) {return ("err")})
            if(!is.data.frame(fit)){
                if(fit == "err"){
                    next
                }
            }
            if(nrow(fit) == 0) next

            fit$cg <- cg[i]
            models.all <- rbind(models.all,fit)
            # EA only
            trainX <- trainX[idx.ea,]
            if(sum(apply(trainX,2,var)!=0) <= 1){
                    next
            }
            trainY <- trainY[idx.ea]
            fit <- tryCatch(
                    elastic.net(trainX,trainY),
                    error = function(e) {return ("err")})
            if(!is.data.frame(fit)){
                if(fit == "err"){
                    next
                }
            }
            if(nrow(fit) == 0) next
            fit$cg <- cg[i]
            models.ea <- rbind(models.ea,fit)
    }
}

1 
[1] "This cg is: 1987413"
[1] "Dim of trainX: 297" "Dim of trainX: 43" 
[1] "Len of trainY: 297"
[1] "Coefficients when fitting: "
                       s1
(Intercept) -4.552958e-05
1977810      7.693460e-04
1981360     -1.063717e-03
1987719     -1.223081e-03
1987813      1.830253e-04
1990232     -1.302585e-03
1994382      1.107820e-03
[1] "Coefficients when optimal: "
                       s0
(Intercept) -4.062561e-05
1977810      7.759077e-04
1981360     -1.070675e-03
1987719     -1.224151e-03
1987813      1.768736e-04
1990232     -1.298076e-03
1994382      1.105434e-03
[1] "Dim of trainX: 133" "Dim of trainX: 43" 
[1] "Len of trainY: 133"
[1] "Coefficients when fitting: "
                      s1
(Intercept) -0.001243466
[1] "Coefficients when optimal: "
                      s0
(Intercept) -0.001243466
2 
[1] "This cg is: 1987418"
[1] "Dim of trainX: 297" "Dim of trainX: 43" 
[1] "Len of trainY: 297"
[1] "Coefficients when fitting: "
                       s1
(Intercept)  0.00

In [70]:
wind[k]

[1] 10000

In [71]:
models.ea <- models.ea[models.ea[,1] != "(Intercept)",]
models.all <- models.all[models.all[,1] != "(Intercept)",]

In [72]:
if(!dir.exists(outd)) dir.create(outd)

In [73]:
head(models.all)

,features,coefs,cg
,<chr>,<dbl>,<dbl>
2,1977810,0.0007759077,1987413
3,1981360,-0.0010706745,1987413
4,1987719,-0.0012241509,1987413
5,1987813,0.0001768736,1987413
6,1990232,-0.0012980759,1987413
7,1994382,0.0011054336,1987413


In [74]:
dim(models.all)

[1] 462   3

In [83]:
# mwas by models of all samples
cg2 <- unique(models.all$cg)
mwas.all <- matrix(0,nrow=length(cg2),ncol=2)
for(i in 1:length(cg2)){
        pos <- models.all[models.all$cg == cg2[i],1]
        gwas <- snp.gwas2$z[is.element(snp.gwas2$pos_hg38, pos)]
        weight <- models.all[models.all$cg == cg2[i],2]
        geno <- snp.1kg.eur2[match(pos,map.1kg.eur2$POS),]
        mwas.all[i,] <- MWAS(gwas, weight, t(geno))
}
rownames(mwas.all) <- cg2
colnames(mwas.all) <- c("z","p")

# mwas by models of EA samples
cg2 <- unique(models.ea$cg)
mwas.ea <- matrix(0,nrow=length(cg2),ncol=2)
for(i in 1:length(cg2)){
        pos <- models.ea[models.ea$cg == cg2[i],1]
        gwas <- snp.gwas2$z[is.element(snp.gwas2$pos_hg38, pos)]
        weight <- models.ea[models.ea$cg == cg2[i],2]
        geno <- snp.1kg.eur2[match(pos,map.1kg.eur2$POS),]
        mwas.ea[i,] <- MWAS(gwas, weight, t(geno))
}
rownames(mwas.ea) <- cg2
colnames(mwas.ea) <- c("z","p")

# output models and mwas results
outf <- paste0(outd,"/models-cov1.all.wind.",wind[k])
write.csv(models.all,outf)
outf <- paste0(outd,"/models-cov1.ea.wind.",wind[k])
write.csv(models.ea,outf)
outf <- paste0(outd,"/mwas-cov1.all.wind.",wind[k])
write.csv(mwas.all,outf)
outf <- paste0(outd,"/mwas-cov1.ea.wind.",wind[k])
write.csv(mwas.ea,outf)

In [84]:
models.all

,features,coefs,cg
,<chr>,<dbl>,<dbl>
2,1977810,0.0007759077,1987413
3,1981360,-0.0010706745,1987413
4,1987719,-0.0012241509,1987413
5,1987813,0.0001768736,1987413
6,1990232,-0.0012980759,1987413
7,1994382,0.0011054336,1987413
9,1977810,0.0007773495,1987418
10,1981360,-0.0010746855,1987418
11,1987719,-0.0013075357,1987418


In [85]:
mwas.all

,z,p
1987413,-9.927849,3.149753e-23
1987418,-9.957674,2.334605e-23
1987434,-9.972340,2.014272e-23
1987449,-9.987919,1.721576e-23
1987458,-10.002652,1.483695e-23
1987468,-10.000385,1.518059e-23
1987492,-10.011781,1.352944e-23
1987511,-10.011545,1.356183e-23
1987527,-10.010634,1.368726e-23
1987565,-10.012276,1.346186e-23


In [76]:
models.all

,features,coefs,cg
,<chr>,<dbl>,<dbl>
2,1977810,0.0007759077,1987413
3,1981360,-0.0010706745,1987413
4,1987719,-0.0012241509,1987413
5,1987813,0.0001768736,1987413
6,1990232,-0.0012980759,1987413
7,1994382,0.0011054336,1987413
9,1977810,0.0007773495,1987418
10,1981360,-0.0010746855,1987418
11,1987719,-0.0013075357,1987418


In [77]:
mwas.all

,z,p
1987413,-9.927849,3.149753e-23
1987418,-9.957674,2.334605e-23
1987434,-9.972340,2.014272e-23
1987449,-9.987919,1.721576e-23
1987458,-10.002652,1.483695e-23
1987468,-10.000385,1.518059e-23
1987492,-10.011781,1.352944e-23
1987511,-10.011545,1.356183e-23
1987527,-10.010634,1.368726e-23
1987565,-10.012276,1.346186e-23


In [78]:
head(models.ea)

,features,coefs,cg
,<chr>,<dbl>,<dbl>
3,1987676,-0.0001675584,1987418
4,1987719,-0.0006830251,1987418
5,1990232,-0.0008377782,1987418
7,1987676,-0.0004591772,1987434
8,1987719,-0.0010073666,1987434
9,1990232,-0.0011692692,1987434
